In [2]:
from collections import namedtuple
import os
os.chdir('../')

In [48]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    'root_dir',
    'source_url',
    'local_data_dir',
    'local_data_file',
    'unzip_dir'
])

In [49]:
from Emotion_Detector.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
from Emotion_Detector.utils.helper import read_yaml, create_directories

In [50]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        create_directories([config.local_data_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_dir = config.local_data_dir,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [51]:
import os
import urllib.request as request
from zipfile import ZipFile

In [52]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )

    def _preprocess(self, zf : ZipFile, f : str, working_dir : str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file = self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            for f in list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [53]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e
